In [1]:
import pandas as pd
import numpy as np
import random 
import torch 
import os 
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torch import nn
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import image
import shutil
from sklearn.mixture import GaussianMixture
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

In [2]:
from BalancingGroups.datasets import * 
import utils_glue

In [4]:
datapath = '/scratch/paa9751/mlhc-project/new_data'
dataloaders = get_loaders(datapath, "multinli", 10, method="erm", duplicates=None)

In [5]:
for x in dataloaders['tr']: #i,x,y,g 
    print('shapes')
    print(x[0].shape,x[1].shape,x[2].shape,x[3].shape)
    print('************************************************')
    print(x[0])
    print('************************************************')
    #print(x[1][x[1][2]==1])
    print('************************************************')
    print(x[2])
    print('************************************************')
    print(x[3])
    break 

shapes
torch.Size([10]) torch.Size([10, 128, 3]) torch.Size([10]) torch.Size([10])
************************************************
tensor([131284,  23117, 198590,  30433,  53495, 126077,  77863, 163609,  26764,
        182149])
************************************************
************************************************
tensor([1, 2, 1, 0, 2, 0, 1, 1, 0, 2])
************************************************
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [149]:
import torch.nn.functional as F 

In [7]:
test=x[1]
shifts=test.argmax(1)[:,2]
shifts=-shifts
batch_size, seq_length, channels = test.shape
arange1 = torch.arange(seq_length)
arange2 = arange1.unsqueeze(0) - shifts.unsqueeze(1)
arange2 = arange2 % seq_length
res = test[torch.arange(batch_size).unsqueeze(1), arange2, :]
mask_zero=torch.unsqueeze(res[:,:,2], dim=2).expand(-1, -1, 3)
res=res*mask_zero

In [183]:
test=x[1]
mask=x[1][:,:,2]==1

for k,batch in enumerate(range(test.shape[0])):
    test_batch = test[batch][mask[batch]]
    test_padded = F.pad(test_batch,(0,0,0,128-test_batch.shape[0]),'constant',0)
    test_padded=test_padded.unsqueeze(0)
    if k==0:
        result = test_padded
    else:
        result = torch.cat((result, test_padded))
    print(result.shape)

torch.Size([1, 128, 3])
torch.Size([2, 128, 3])
torch.Size([3, 128, 3])
torch.Size([4, 128, 3])
torch.Size([5, 128, 3])


In [169]:
test0=test[0][mask[0]]
(F.pad(test0,(0,0,0,128-test0.shape[0]),'constant',0)).shape

torch.Size([128, 3])

In [75]:
mask=(x[1][:,:,2]==1).int()
# mask=mask.unsqueeze(2)
# mask=mask.expand(-1,-1,3)
# x[1][mask].shape
mask=mask.unsqueeze(-1)

mask.shape

torch.Size([5, 128, 1])

In [76]:
masked_tensor = x[1][mask].reshape(-1, 3)

In [77]:
counts = mask.sum(axis=1)
result = np.split(masked_tensor, np.cumsum(counts[:-1]))

In [91]:
counts

tensor([[ 7],
        [14],
        [19],
        [15],
        [10]])

In [16]:
metadata=pd.read_csv('new_data/multinli/data/metadata_random.csv')
metadata_random = pd.read_csv('new_data/metadata_multinli.csv')

In [17]:
metadata

,Unnamed: 0,gold_label,sentence2_has_negation,split
0,0,2,0,2
1,1,1,0,0
2,2,1,0,2
3,3,1,0,2
4,4,2,0,0
...,...,...,...,...
412344,412697,0,0,1
412345,412698,0,0,0
412346,412699,0,0,1
412347,412700,2,0,2


In [18]:
metadata_random

,id,filename,split,y,a
0,0,0,2,2,0
1,1,1,0,1,0
2,2,2,2,1,0
3,3,3,2,1,0
4,4,4,0,2,0
...,...,...,...,...,...
412344,412344,412344,1,0,0
412345,412345,412345,0,0,0
412346,412346,412346,1,0,0
412347,412347,412347,2,2,0


In [95]:
test =x[1].clone()

In [97]:
mask = test[:,:,2]==1
mask.shape

torch.Size([5, 128])

In [103]:
mask2= mask.unsqueeze(-1)

In [104]:
mask2.shape

torch.Size([5, 128, 1])

In [106]:
mask3 = mask2.expand(test.size())
mask3.shape

torch.Size([5, 128, 3])

In [113]:
test[mask3].shape

torch.Size([195])

In [115]:
mask.sum(axis=1)

tensor([ 7, 14, 19, 15, 10])

In [126]:
result = (test*mask3)
result[0]

tensor([[    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [ 2833,     1,     1],
        [22623,     1,     1],
        [25537,     1,     1],
        [ 1996,     1,     1],
        [ 2103,     1,     1],
        [ 1012,     1,     1],
        [  102,     1,     1],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        [    0,     0,     0],
        

In [141]:
mask_result  = (result!=0).int()
shifts = torch.argmax(mask_result[0], 0)

tensor([5, 5, 5])

In [135]:
mask_result[0]

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 

In [2]:
## change chexpert

In [196]:
chexdf20k = pd.read_csv('/scratch/paa9751/mlhc-project/resized_data/chexpert/full_data_chexpert_newdata.csv')

In [197]:
new_paths=[]
for i,row in chexdf20k.iterrows():
    img_path = '/'+row['Path']
    path_name = "".join(row['Path'].split('/'))[:-4]
    dest_path = os.path.join('/scratch/paa9751/mlhc-project/resized_data/chexpert', "imgs/"+path_name+".npy")
    new_paths.append(dest_path)
chexdf20k['new_path'] = new_paths

In [198]:
chexdf20k['dataset_idx']=1
chexdf20k['true_group_idx'] = 1 - chexdf20k.Cardiomegaly
chexdf20k=chexdf20k[['subject_id','Cardiomegaly','new_path','split','dataset_idx','true_group_idx']]
chexdf20k['old_split'] = chexdf20k.split
chexdf20k['split'] = ['none']*chexdf20k.shape[0]

In [199]:
def get_new_split(data_df):
    newidx={}
    for split in ['train','val']:
        split_df = data_df[data_df.old_split==split]
        df1 = split_df[split_df['Cardiomegaly'] == 1]
        df0 = split_df[split_df['Cardiomegaly'] == 0]
        p,n=0.5,0.5
        # if split=='test':
        #     split_df = data_df[data_df.sp=='none']
        #     p=0.9 * 0.5
        #     n=0.1 * 0.5
        n1,n0 = int(p* df1.shape[0]), int(n * df0.shape[0])  # 90% of K
        n1_,n0_ = random.sample(list(df1.index),n1), random.sample(list(df0.index),n0)
        newidx[split]=n1_+n0_
    return newidx

newidx=get_new_split(chexdf20k)
chexdf20k.loc[newidx['train'],'split'] = 'train'
chexdf20k.loc[newidx['val'],'split'] = 'val'

In [200]:
split_df = chexdf20k[chexdf20k.split=='none']
df1 = split_df[split_df['Cardiomegaly'] == 1]
df0 = split_df[split_df['Cardiomegaly'] == 0]
p,n=0.5,0.5
n1,n0 = 95, 850  # 90% of K
n1_,n0_ = random.sample(list(df1.index),n1), random.sample(list(df0.index),n0)
testidx=n1_+n0_
chexdf20k.loc[testidx,'split'] = 'test'

In [201]:
chexdf20k.split.value_counts()

split
none     7555
train    7000
val      1500
test      945
Name: count, dtype: int64

In [202]:
chexdf20k.shape

(17000, 7)

In [203]:
chexdf20k.to_csv('/scratch/paa9751/mlhc-project/resized_data/chexpert/full_data_chexpert_full_new.csv')

In [2]:
chexdf10k= pd.read_csv('/scratch/paa9751/mlhc-project/resized_data/chexpert/full_data_chexpert_full.csv')

In [4]:
chexdf10k.old_path

0       CheXpert-v1.0/train/patient17799/study2/view2_...
1       CheXpert-v1.0/train/patient34816/study6/view1_...
2       CheXpert-v1.0/train/patient34722/study10/view1...
3       CheXpert-v1.0/train/patient51325/study1/view1_...
4       CheXpert-v1.0/train/patient40103/study7/view1_...
                              ...                        
9995    CheXpert-v1.0/train/patient27549/study18/view1...
9996    CheXpert-v1.0/train/patient34539/study1/view2_...
9997    CheXpert-v1.0/train/patient34141/study16/view3...
9998    CheXpert-v1.0/train/patient30393/study6/view2_...
9999    CheXpert-v1.0/train/patient05112/study1/view2_...
Name: old_path, Length: 10000, dtype: object

In [184]:
chexdf10k[chexdf10k.split=='test'].Cardiomegaly.value_counts()

Cardiomegaly
0.0    1350
1.0     150
Name: count, dtype: int64

In [97]:
chexdf10k[chexdf10k.split=='train'].Cardiomegaly.value_counts()

Cardiomegaly
1.0    6300
0.0     700
Name: count, dtype: int64

In [26]:
chexdf10k.head()

,Unnamed: 0.1,Unnamed: 0,subject_id,Cardiomegaly,old_path,new_path,split,dataset_idx,true_group,true_group_idx,...,sns_group_5,max_accuracy_model_predictions,min_loss_model_predictions,predprob_new,sns_y_2,sns_y_3,sns_y_4,sns_y_6,sns_y_8,sns_y_16
0,0,0,patient17799,1.0,CheXpert-v1.0/train/patient17799/study2/view2_...,/scratch/paa9751/mlhc-project/resized_data/che...,train,1,11,0,...,0,1.940106,1.940106,0.874364,0,0,0,0,0,0
1,1,1,patient34816,1.0,CheXpert-v1.0/train/patient34816/study6/view1_...,/scratch/paa9751/mlhc-project/resized_data/che...,train,1,11,0,...,2,3.293948,3.293948,0.964221,0,1,1,1,1,1
2,2,2,patient34722,0.0,CheXpert-v1.0/train/patient34722/study10/view1...,/scratch/paa9751/mlhc-project/resized_data/che...,train,1,1,1,...,4,5.707192,5.707192,0.996689,1,2,2,2,2,2
3,3,3,patient51325,1.0,CheXpert-v1.0/train/patient51325/study1/view1_...,/scratch/paa9751/mlhc-project/resized_data/che...,train,1,11,0,...,2,0.869545,0.869545,0.704651,2,0,3,3,3,3
4,4,4,patient40103,1.0,CheXpert-v1.0/train/patient40103/study7/view1_...,/scratch/paa9751/mlhc-project/resized_data/che...,train,1,11,0,...,2,1.718483,1.718483,0.847933,0,0,0,0,0,4


In [3]:
boolist=pd.Series([os.path.isfile(p) for p in chexdf10k.new_path])

In [4]:
boolist.value_counts()

True    10000
Name: count, dtype: int64

In [6]:
chex = pd.read_csv('/scratch/paa9751/mlhc-project/resized_data/chexpert/full_data_chexpert_full.csv').iloc[:,1:]
mimic = pd.read_csv('/scratch/paa9751/mlhc-project/resized_data/mimic/full_data_mimic_full.csv').iloc[:,1:]
df=pd.concat([chex,mimic])

In [10]:
df['weight']=1/df.observed_prob

In [15]:
df['clipped_weight'] = [20 if w>20 else w for w in df.weight]

In [17]:
df.clipped_weight

0        1.143689
1        1.037107
2       20.000000
3        1.419142
4        1.179338
          ...    
9995    14.624398
9996     1.259605
9997     4.548131
9998     1.472992
9999     2.612550
Name: clipped_weight, Length: 20000, dtype: float64

In [21]:
df[df.dataset_idx==1].to_csv('/scratch/paa9751/mlhc-project/resized_data/chexpert/full_data_chexpert_full.csv')
df[df.dataset_idx==0].to_csv('/scratch/paa9751/mlhc-project/resized_data/mimic/full_data_mimic_full.csv')